<a href="https://colab.research.google.com/github/swapnilbetkar/python/blob/Dev_Self/DecisionTree_Lect2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>